<a href="https://colab.research.google.com/github/jamesshastry/gpu/blob/main/flops.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Wed Feb  8 22:26:15 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   63C    P0    26W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import torch
from torch.profiler import profile, ProfilerActivity

'''
Achieved FLOPS for various numerical operations.
'''

def do_work():
    SIZE = 2**12
    ones = torch.ones((SIZE, SIZE), device=torch.device('cuda'))
    
    torch.matmul(ones, ones, out=ones)

    # In place multiplication
    ones.mul_(0.5) 
    
    result = ones.mul(0.5)
    
    total = ones + result
    
    result = torch.sum(ones)

    # sqrt takes 7 ops.
    result = torch.sqrt(ones)
    
    # sin takes 17 ops (14 fp64, 3 fp32).
    result = torch.sin(ones) 
    
    # sigmoid takes 24 ops.
    result = torch.sigmoid(ones)
    
    # log10 takes 24 ops.
    result = torch.log10(ones)
    
    # pow takes 142 ops.
    result = torch.pow(ones, 3.14159)

do_work()

with profile(  
    activities = [ProfilerActivity.CPU, ProfilerActivity.CUDA],
    profile_memory = True,
    record_shapes = True
    #with_stack = True
) as prof:

    do_work()

filename = f"./flops_with_warmup.json"
prof.export_chrome_trace(filename)